In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
healthfc_df = pd.read_csv("healthFC_annotated.csv")
hfc_questions = healthfc_df.en_claim.tolist()
hfc_labels = healthfc_df.label.tolist()


In [ ]:
import json

with open("BioASQ-train-yesno-7b.json", "r") as f:
    bioasq_content = json.load(f)

bioasq_questions = list()
bioasq_answers = list()
for qid in range(len(bioasq_content["data"][0]["paragraphs"])):
    question = bioasq_content["data"][0]["paragraphs"][qid]["qas"][0]["question"]
    answer = bioasq_content["data"][0]["paragraphs"][qid]["qas"][0]["answers"]
    if question not in bioasq_questions:
        bioasq_questions.append(question)
        bioasq_answers.append(answer)


In [ ]:
!pip install retriv
!pip install dill
!pip install torch
!pip install sentence_transformers
!pip install scikit-learn
import nltk
nltk.download('punkt')

txt file gen. After this code, merge it with true_label given in datasets

In [ ]:
import dill as pickle
from retriv import SparseRetriever

sr = SparseRetriever(
  index_name="pubmed-index",
  model="bm25",
  min_df=10,
  tokenizer="whitespace",
  stemmer="english",
  stopwords="english",
  do_lowercasing=True,
  do_ampersand_normalization=True,
  do_special_chars_normalization=True,
  do_acronyms_normalization=True,
  do_punctuation_removal=True,
)

corpus_path = "/content/drive/MyDrive/pubmed_landscape_abstracts.csv"

import time
start = time.time()

sr = sr.index_file(
  path=corpus_path,
  show_progress=True,
  callback=lambda doc: {
    "id": doc["PMID"],
    "text": doc["AbstractText"],
    }
  )

with open('/content/drive/MyDrive/Tesi/Prove_codice_/pickled1_sr', 'wb') as file:
    pickle.dump(sr, file)

In [ ]:
import pickle
import torch
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import nltk


nltk.download('punkt')


start_idx = 0  # start
end_idx = 1048  # end


with open('/content/drive/MyDrive/pickled_sr', 'rb') as file:
    inverted_index = pickle.load(file)


df = pd.read_csv("/content/drive/MyDrive/healthFC_annotated.csv")
claims = df.en_claim.tolist()
labels = df.label.tolist()


query_list = []
for idx, claim in enumerate(claims[start_idx:end_idx]):
    claim = claim.lower()
    query_list.append({"id": str(idx + start_idx), "text": claim})


results = inverted_index.msearch(
  queries=query_list,
  cutoff=20,
)

# Carica il file CSV degli abstract
abstracts = pd.read_csv("/content/drive/MyDrive/abstract_merged.csv")

# Crea un dizionario di mapping dagli ID degli abstract ai loro testi
abstract_dict = pd.Series(abstracts.AbstractText.values, index=abstracts.PMID.astype(str)).to_dict()

# Rimappa tutti gli ID dei documenti dai risultati.
remapped_all_ids = []
for claim_id, document_results in results.items():
    remapped_ids = []
    for doc_id, score in document_results.items():
        if doc_id in abstract_dict:
            remapped_ids.append(doc_id)
        else:
            print(f"Skipping invalid doc_id: {doc_id}")
    remapped_all_ids.append(remapped_ids)

claim_sentences = []
for ids in remapped_all_ids:
    all_sentences = []
    for doc_id in ids:
        abstract = abstract_dict[doc_id]
        sentences = sent_tokenize(abstract)
        all_sentences.append((doc_id, sentences))
    claim_sentences.append(all_sentences)

# Carica il modello di trasformatore di frasi per selezionare le frasi di prova.
model = SentenceTransformer('copenlu/spiced')
print("loaded sentence model!")

# Trova le prime 3 frasi più simili alla query da ciascun documento, per condensarlo nelle parti più importanti.
top_sentences = []
for idx in range(start_idx, min(end_idx, len(claims))):
    claim = claims[idx]
    print(f"Processing claim {idx + 1}/{len(claims)}")

    if idx - start_idx >= len(claim_sentences):
        print(f"Warning: claim index {idx} out of range")
        continue

    sents_per_doc = claim_sentences[idx - start_idx]

    doc_top_sentences = []
    for doc_id, sents in sents_per_doc:
        sents_embeddings = model.encode(sents, convert_to_tensor=True)
        claim_embedding = model.encode(claim, convert_to_tensor=True)
        cos_scores = util.cos_sim(claim_embedding, sents_embeddings)[0]

        new_k = 3
        if len(sents) < 3:
            new_k = len(sents)

        top_results = torch.topk(cos_scores, k=new_k)

        np_results = top_results[1].detach().cpu().numpy()
        np_results = np_results[np_results < len(sents)]  # Filtra gli indici fuori range
        doc_top_sentences.append((doc_id, np_results))
    top_sentences.append(doc_top_sentences)

selected_sentences = []
for doc_sents, doc_top_sents in zip(claim_sentences, top_sentences):
    doc_selected_sents = []
    for (doc_id, sents), (top_doc_id, top) in zip(doc_sents, doc_top_sents):
        selected_sents = np.array(sents)[top]
        doc_selected_sents.append((doc_id, selected_sents))
    selected_sentences.append(doc_selected_sents)

# Crea una lista congiunta di claims concatenati e prove, in forma di "claim [SEP] evidence1 evidence2 evidence3"
joint_list = []
for idx in range(start_idx, min(end_idx, len(claims))):
    claim = claims[idx]
    for doc_id, sents in selected_sentences[idx - start_idx]:
        joint = claim + " [SEP] "
        for s in sents:
            joint += s + f" [{doc_id}] "
        joint_list.append(joint)

output_filename = f"/content/drive/MyDrive/20_sentences_SciFact.txt"
with open(output_filename, "w") as f:
    for line in joint_list:
        f.write(line)
        f.write("\n")

with open(output_filename, "r") as f:
    lines = f.readlines()
    flatlines = []
    for l in lines:
        l = l.strip()
        if "[SEP]" in l:
            flatlines.append(l)

print("Processing complete!")

In [ ]:
import csv
from collections import defaultdict

# Leggere il file di testo
file_path = '/content/drive/MyDrive/20_sentences_healthFC.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Dizionario per raggruppare le frasi e le relative sentence
frase_dict = defaultdict(list)

# Parsing delle linee
for line in lines:
    if '[SEP]' in line:
        # Separare la frase e la sentence
        frase, sentence = line.split('[SEP]')
        sentence = sentence.strip()

        # Aggiungere la sentence al dizionario
        frase_dict[frase.strip()].append(sentence)

# Scrittura nel file CSV
csv_file_path = '/content/drive/MyDrive/Health_fc_sentences_true_label.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Scrivere l'intestazione
    csvwriter.writerow(['Frase', 'Sentences Concatenate'])

    # Scrivere i dati raggruppati
    for frase, sentences in frase_dict.items():
        sentences_concat = ' '.join(sentences)
        csvwriter.writerow([frase, sentences_concat])

print(f"File CSV generato: {csv_file_path}")

Dense

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
from torch.utils.data import DataLoader
import logging

# Imposta logging per monitorare il processo
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# File paths
corpus_path = "/content/drive/MyDrive/Scifact_elaborazione/corpus_train.csv"
questions_path = "/content/drive/MyDrive/Scifact_elaborazione/claims_train.csv"
output_path = "retrieval_results_dense_scifact.csv"

# Caricamento dei dati
logging.info("Caricamento dei file CSV...")
corpus = pd.read_csv(corpus_path)
queries = pd.read_csv(questions_path)

# Assicurati che le colonne esistano
assert 'abstract' in corpus.columns, "Il corpus deve avere una colonna 'abstract'."
assert 'claim' in queries.columns, "Il file delle query deve avere una colonna 'claim'."

# Prepara i documenti e le query
documents = corpus['abstract'].dropna().tolist()
claims = queries['claim'].dropna().tolist()

# Controllo dei documenti
if not documents:
    raise ValueError("La lista dei documenti è vuota.")
if not claims:
    raise ValueError("La lista delle query è vuota.")

# Verifica della GPU
if not torch.cuda.is_available():
    logging.error("GPU non disponibile. Assicurati di avere una GPU configurata correttamente.")
    raise EnvironmentError("GPU non disponibile.")
device = 'cuda'

# Caricamento del modello SBERT
logging.info("Caricamento del modello SBERT...")
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)

# Funzione per codificare i documenti in batch
def encode_in_batches(model, documents, batch_size=32):
    logging.info("Codifica dei documenti in batch...")
    dataloader = DataLoader(documents, batch_size=batch_size)
    embeddings = []
    for batch in dataloader:
        batch_embeddings = model.encode(batch, convert_to_tensor=True, device=device)
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings)

# Codifica dei documenti
logging.info("Codifica dei documenti...")
document_embeddings = encode_in_batches(model, documents)

# Lista per i risultati finali
results = []

# Iterazione sulle query
logging.info("Avvio del retrieval denso...")
for idx, query in enumerate(claims):
    logging.info(f"Processamento query {idx + 1}/{len(claims)}: {query}")

    # Codifica della query
    query_embedding = model.encode(query, convert_to_tensor=True, device=device)

    # Calcolo dei punteggi di similarità
    dense_scores = util.cos_sim(query_embedding, document_embeddings).squeeze().tolist()

    # Ordina i documenti in base ai punteggi
    ranked_indices = np.argsort(dense_scores)[::-1]
    ranked_docs = [(documents[i], dense_scores[i]) for i in ranked_indices]

    # Salva i top 5 risultati per la query
    for rank, (doc, score) in enumerate(ranked_docs[:5], start=1):
        results.append({
            'query': query,
            'document': doc,
            'score': score,
            'rank': rank
        })

# Salva i risultati in un file CSV
logging.info("Salvataggio dei risultati in un file CSV...")
results_df = pd.DataFrame(results)
results_df.to_csv(output_path, index=False)

logging.info(f"Processo completato. Risultati salvati in: {output_path}")


CSV CREATION WITH CLAIM, SENTENCES AND TRUE LABEL (given in the dataset folder)

1.   BioASQ_sentences_true_label.csv
2.   Health_fc_sentences_true_label.csv


In [ ]:
import pandas as pd

# Percorso del file CSV
csv_file_path = '/content/drive/MyDrive/BioASQ_sentences_true_label.csv'

# Leggere il file CSV
df = pd.read_csv(csv_file_path)

In [ ]:
import pandas as pd

# Percorso del file CSV
csv_file_path = '/content/drive/MyDrive/Health_fc_sentences_true_label.csv'

# Leggere il file CSV
df = pd.read_csv(csv_file_path)

In [ ]:
!pip install datasets pandas sentence_transformers
!pip install -U transformers
# Install below if using GPU
!pip install accelerate
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers
import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random
from sentence_transformers import SentenceTransformer
import pandas as pd

fireworks.client.api_key = ""
dotenv.load_dotenv()

mixtral_llm = "mixtral-8x22b-instruct"

def get_completion(prompt, model=None, max_tokens=50):

    fw_model_dir = "accounts/fireworks/models/"

    model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0.3
    )

    return completion.choices[0].text

Fireworks.ai version

In [ ]:
import pandas as pd
import re

def add_to_dataframe(query, Risposta):
    # Trova la label nella risposta usando una regex
    label_match = re.search(r"Label: (\w+)", Risposta)
    if label_match:
        label = label_match.group(1)  # Estrai la label
        justification = Risposta[label_match.end():].strip()  # Tutto ciò che segue la label è la justification
        claim = query.strip()  # Il claim è semplicemente la query data
        new_row = pd.DataFrame([[claim, label, justification]], columns=["Claim", "Predicted label", "Justification"])
        global df1
        df1 = pd.concat([df1, new_row], ignore_index=True)

start_index = 1  # Modifica questo valore a tuo piacimento

num_rows = 2 # Numero di righe da considerare

for index in range(start_index, start_index + num_rows):
    query = df.loc[index, 'Frase']
    results = df.loc[index, 'Sentences Concatenate']

    # Costruisci il prompt template con query e results prelevati
    prompt_template = f'''[INST]  <<SYS>>

    You are a helpful, respectful and honest Doctor. Always answer as helpfully as possible using the context text provided. Try to give an explanation based on {results} to the question {query}. elaborate the context to generate a new information.

    Use only the knowledge in results to answer.

    Answer describing in a scentific way. Be formal during the answer. Use the third person.

    Answer without mentioning the context. Use it but don't refer to it in the text

    to answer, use max 200 word

    with the answer, gives a line like: "Label:". Always put Label as first. After Label, give the justification
    Label can be yes, no, NEI, where yes: claim is true. no: claim is false. NEI: not enough information.
    <<SYS>>

    """CONTEXT:/n/n {results}/n

    Question: {query} [/INST]

    '''

    responses = get_completion(prompt_template, model=mixtral_llm, max_tokens=400)
    Risposta = ''.join([str(r) for r in responses])

    # Stampa la risposta per ogni iterazione
    print("")
    print(Risposta)
    try:
      df1
    except NameError:
      df1 = pd.DataFrame(columns=["Claim", "Predicted label", "Justification"])
    # Esempio di utilizzo
    add_to_dataframe(query, Risposta)

Datasets with Justification

1.   HealthFc_Frase_Predicted label_Justification_Sentences Concatenate_True label.csv
2.   BioASQ_Frase_Predicted label_Justification_Sentences Concatenate_True label.csv



In [ ]:
import pandas as pd

# Definire il dizionario di mapping inverso
label_mapping_inverse = {0: 'Yes', 1: 'NEI', 2: 'No'}

# Applicare il mapping inverso alla colonna 'True label'
selected_columns['True label'] = selected_columns['True label'].map(label_mapping_inverse)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcolo delle metriche
precision = precision_score(selected_columns['True label'], selected_columns['Predicted label'], average='macro')
recall = recall_score(selected_columns['True label'], selected_columns['Predicted label'], average='macro')
f1 = f1_score(selected_columns['True label'], selected_columns['Predicted label'], average='macro')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
#For single class
from sklearn.metrics import precision_score

# Definizione delle etichette
labels = ['Yes', 'NEI', 'No']

# Calcolo delle precisioni per ogni classe
precision_per_class = precision_score(selected_columns['True label'], selected_columns['Predicted label'], labels=labels, average=None)

# Stampare le precisioni per ogni classe
for label, precision in zip(labels, precision_per_class):
    print(f'Precision for class {label}: {precision:.4f}')

Classification with MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33 to avoid allucination

Always load the right csv before

In [ ]:
import pandas as pd

# Percorso del file CSV
csv_file_path = '/content/drive/MyDrive/BioASQ_Frase_Predicted_label_Justification_Sentences Concatenate_True label.csv'

# Leggere il file CSV
df = pd.read_csv(csv_file_path)

In [ ]:
import pandas as pd
from transformers import pipeline

# Set start and end indices
start_idx = 0
end_idx = 749

# Configure the zero-shot classifier
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33")

# Function to generate justifications
def generate_justification(row):
    claim = row['Frase']
    sentences=['Sentence_concatenate']
    justifications = row["Justification"]
    true_label = row['True label']
    predicted_label = row['Predicted label']
    Class = ["True", "False", "NEI"]
    justification1 = f'"prediction: {predicted_label}" for: justificatoin used for the prediction "{justifications}" with sentencies {sentences}'
    max_length = 512 # Some buffer for tokens
    if len(justification1) > max_length:
        justification1 = justification1[:max_length]
    output = zeroshot_classifier(claim, Class, hypothesis_template="The claim is '{}' for: " + justification1, multi_label=False)

    final_justification = f'{output}. Predicted label was {predicted_label}. True label was {true_label}'
    return final_justification

# Iterate through the DataFrame rows and generate justifications
file_path = "/content/drive/MyDrive/output_senza_sentences.txt"

for idx in range(start_idx, end_idx + 1):
    row = result.iloc[idx]
    validità = generate_justification(row)
    output_string = f"row {idx}: {validità}"
   # print(output_string)

    # Write the result to the file immediately
    with open(file_path, 'a') as file:
        file.write(output_string + "\n")
        print("stringa ")
        print(idx)

Results.txt for this phase

In [ ]:
import pandas as pd
import re

# Funzione per estrarre le informazioni da una riga di testo
def parse_line(line):
    sequence_match = re.search(r"'sequence': '(.*?)'", line)
    label_match = re.search(r"'labels': \['(.*?)', '(.*?)'\]", line)
    true_label_match = re.search(r'True label was (Yes|No)', line)

    sequence = sequence_match.group(1) if sequence_match else ''
    label = label_match.group(1) if label_match else ''
    true_label = true_label_match.group(1) if true_label_match else ''

    return sequence, label, true_label

# Leggi il file di testo
input_file = '/content/drive/MyDrive/output.txt' #Change here to read the results (FOR BIOASQ RES)

data = []

with open(input_file, 'r') as infile:
    for line in infile:
        sequence, label, true_label = parse_line(line)
        data.append([sequence, label, true_label])

# Crea un DataFrame
df = pd.DataFrame(data, columns=['sequence', 'label', 'True Label'])

# Mostra il DataFrame
print(df)

For health_fc

In [ ]:
import pandas as pd
import re

# Funzione per estrarre le informazioni da una riga di testo
def parse_line(line):
    sequence_match = re.search(r"'sequence': '(.*?)'", line)
    labels_match = re.search(r"'labels': \[(.*?)\]", line)
    true_label_match = re.search(r'True label was (\d+)', line)
    predicted_label_match = re.search(r'Predicted label was (\w+)', line)

    sequence = sequence_match.group(1) if sequence_match else ''
    labels = labels_match.group(1).replace("'", "").split(', ') if labels_match else []
    true_label = int(true_label_match.group(1)) if true_label_match else -1
    predicted_label = predicted_label_match.group(1) if predicted_label_match else ''

    return sequence, labels, true_label, predicted_label

# Leggi il file di testo
input_file = '/content/drive/MyDrive/output.txt'

data = []

with open(input_file, 'r') as infile:
    for line in infile:
        sequence, labels, true_label, predicted_label = parse_line(line)
        data.append([sequence, labels, true_label, predicted_label])

# Crea un DataFrame
df = pd.DataFrame(data, columns=['sequence', 'labels', 'True Label', 'Predicted Label'])

# Mostra il DataFrame
print(df)

In [ ]:
# Calcolare precision, recall e F1-score
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(df['True Label'], df['Predicted Label'], pos_label='Yes', average='weighted')
recall = recall_score(df['True Label'], df['Predicted Label'], pos_label='Yes', average='weighted')
f1 = f1_score(df['True Label'], df['Predicted Label'], pos_label='Yes', average='weighted')

# Visualizzare i risultati
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Fine_tuning

In [ ]:
!pip install transformers datasets torch
!pip install scikit-learn pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

Code to split in train, validation and test set

In [ ]:
!from sklearn.model_selection import train_test_split

# Prima, suddividiamo in train+validation e test
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['True Label'], random_state=42)

# Poi, suddividiamo train+validation in train e validation
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['True Label'], random_state=42)
# Nota: 0.25 * 0.8 = 0.2, quindi il set di validazione sarà il 20% del totale

In [ ]:
print(df_merged['True Label'].value_counts())
from sklearn.model_selection import train_test_split

# Suddivisione train+validation e test
train_val_df, test_df = train_test_split(df_merged, test_size=0.2, stratify=df['True Label'], random_state=42)

# Suddivisione train+validation in train e validation
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['True Label'], random_state=42)
# Verifica della distribuzione
print("Distribuzione delle etichette nel training set:")
print(train_df['True Label'].value_counts())

print("Distribuzione delle etichette nel validation set:")
print(val_df['True Label'].value_counts())

print("Distribuzione delle etichette nel test set:")
print(test_df['True Label'].value_counts())

In [ ]:
file_path = '/content/drive/MyDrive/Fine_Tuning/train1.csv'
file_path1 = '/content/drive/MyDrive/Fine_Tuning/val1.csv'
file_path2 = '/content/drive/MyDrive/Fine_Tuning/test1.csv'
# Salva il DataFrame in un file CSV
train_df.to_csv(file_path, index=False)
val_df.to_csv(file_path1, index=False)
test_df.to_csv(file_path2, index=False)
print(f"DataFrame salvato con successo in: {file_path}")


BIOASQ
1.   train
2.   val
3.   test

HealthFC
1.   train1
2.   val1
3.   test1

In [ ]:
file_path = '/content/drive/MyDrive/Fine_Tuning/train1.csv'
file_path1 = '/content/drive/MyDrive/Fine_Tuning/val1.csv'
file_path2 = '/content/drive/MyDrive/Fine_Tuning/test1.csv'
#train1 sono 3 classi
train_df = pd.read_csv(file_path)
val_df = pd.read_csv(file_path1)
test_df = pd.read_csv(file_path2)

In [ ]:
BIOASQ

In [ ]:
import pandas as pd
import numpy as np  # Importare NumPy
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Convertire i DataFrame in Dataset di Hugging Face
def convert_to_dataset(df):
    return Dataset.from_pandas(df)

# Supponiamo che train_df, val_df, e test_df siano già definiti
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

# Creare un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Preprocessare e tokenizzare
model_name = "MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    # Concatenare Claim e Justification e mappare le etichette
    texts = [claim + " " + justification for claim, justification in zip(examples['Claim'], examples['Justification'])]
    labels = [1 if label == 'Yes' else 0 for label in examples['True label']]
    encodings = tokenizer(texts, padding="max_length", truncation=True)
    encodings['labels'] = labels
    return encodings

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Configurare il modello
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Definire gli argomenti di training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.20,
    gradient_accumulation_steps=8,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch"
)

# Definire la funzione di metriche
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision = precision_score(p.label_ids, preds)
    recall = recall_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds)
    return {'precision': precision, 'recall': recall, 'f1': f1}

# Creare il Trainer con specifica della funzione di perdita
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=compute_metrics
)

# Addestrare il modello
trainer.train()

# Valutazione del modello sul set di test
metrics = trainer.evaluate(encoded_dataset['test'])
print("Test set evaluation metrics:", metrics)


HealthFC (Bert Uncased)

In [ ]:
import pandas as pd
import numpy as np  # Importare NumPy
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import shutil
import os

# Percorso della cartella
directory = './results'

# Cancellare tutti i file e le sottocartelle
if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Tutti i contenuti della cartella '{directory}' sono stati cancellati.")
else:
    print(f"La cartella '{directory}' non esiste.")

# Ricreare la cartella (opzionale)
os.makedirs(directory, exist_ok=True)
print(f"La cartella '{directory}' è stata ricreata.")

# Convertire i DataFrame in Dataset di Hugging Face
def convert_to_dataset(df):
    return Dataset.from_pandas(df)

# Supponiamo che train_df, val_df, e test_df siano già definiti
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

# Creare un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Preprocessare e tokenizzare
#model_name = "MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33"
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    texts = [claim + " " + justification for claim, justification in zip(examples['Claim'], examples['Justification'])]
    labels = [0 if label == 'Yes' else 1 if label == 'NEI' else 2 for label in examples['True Label']]
    encodings = tokenizer(texts, padding="max_length", truncation=True)
    encodings['labels'] = labels
    return encodings

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Configurare il modello
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Definire gli argomenti di training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.20,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch"
)

# Definire la funzione di metriche
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision = precision_score(p.label_ids, preds, average='weighted')
    recall = recall_score(p.label_ids, preds, average='weighted')
    f1 = f1_score(p.label_ids, preds, average='weighted')
    accuracy = accuracy_score(p.label_ids, preds)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': accuracy}
# Creare il Trainer con specifica della funzione di perdita
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=compute_metrics
)

# Addestrare il modello
trainer.train()

# Valutazione del modello sul set di test
metrics = trainer.evaluate(encoded_dataset['test'])
print("Test set evaluation metrics:", metrics)

### Try with PubmedBert fine tuned

Healthfc

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Fine_Tuning/train1.csv'
file_path1 = '/content/drive/MyDrive/Fine_Tuning/val1.csv'
file_path2 = '/content/drive/MyDrive/Fine_Tuning/test1.csv'
#train1 sono 3 classi
train_df = pd.read_csv(file_path)
val_df = pd.read_csv(file_path1)
test_df = pd.read_csv(file_path2)

In [ ]:
pip install datasets

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import shutil
import os

# Percorso della cartella
directory = './results'

# Cancellare tutti i file e le sottocartelle
if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Tutti i contenuti della cartella '{directory}' sono stati cancellati.")
else:
    print(f"La cartella '{directory}' non esiste.")

# Ricreare la cartella (opzionale)
os.makedirs(directory, exist_ok=True)
print(f"La cartella '{directory}' è stata ricreata.")

# Convertire i DataFrame in Dataset di Hugging Face
def convert_to_dataset(df):
    return Dataset.from_pandas(df)

# Supponiamo che train_df, val_df, e test_df siano già definiti
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

# Creare un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Preprocessare e tokenizzare
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    texts = [
        (claim if claim is not None else "") + " " + (justification if justification is not None else "")
        for claim, justification in zip(examples['Claim'], examples['Justification'])
    ]
    labels = [0 if label == 'Yes' else 1 if label == 'NEI' else 2 for label in examples['True Label']]

    # Specifica la lunghezza massima
    encodings = tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    encodings['labels'] = labels
    return encodings

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Configurare il modello
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Definire gli argomenti di training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.20,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch"
)

for param in model.parameters():
    param.data = param.data.contiguous()

# Definire la funzione di metriche
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision = precision_score(p.label_ids, preds, average='weighted')
    recall = recall_score(p.label_ids, preds, average='weighted')
    f1 = f1_score(p.label_ids, preds, average='weighted')
    accuracy = accuracy_score(p.label_ids, preds)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': accuracy}

# Creare il Trainer con specifica della funzione di perdita
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=compute_metrics
)

# Addestrare il modello
trainer.train()

# Valutazione del modello sul set di test
metrics = trainer.evaluate(encoded_dataset['test'])
print("Test set evaluation metrics:", metrics)

BioASQ

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Fine_Tuning/train.csv'
file_path1 = '/content/drive/MyDrive/Fine_Tuning/val.csv'
file_path2 = '/content/drive/MyDrive/Fine_Tuning/test.csv'
#train1 sono 3 classi
train_df = pd.read_csv(file_path)
val_df = pd.read_csv(file_path1)
test_df = pd.read_csv(file_path2)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import shutil
import os

# Percorso della cartella
directory = './results'

# Cancellare tutti i file e le sottocartelle
if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Tutti i contenuti della cartella '{directory}' sono stati cancellati.")
else:
    print(f"La cartella '{directory}' non esiste.")

# Ricreare la cartella (opzionale)
os.makedirs(directory, exist_ok=True)
print(f"La cartella '{directory}' è stata ricreata.")

# Convertire i DataFrame in Dataset di Hugging Face
def convert_to_dataset(df):
    return Dataset.from_pandas(df)

# Supponiamo che train_df, val_df, e test_df siano già definiti
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset(test_df)

# Creare un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Preprocessare e tokenizzare
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    texts = [
        (claim if claim is not None else "") + " " + (justification if justification is not None else "")
        for claim, justification in zip(examples['Claim'], examples['Justification'])
    ]
    labels = [0 if label == 'Yes' else 1 for label in examples['True label']]

    # Specifica la lunghezza massima
    encodings = tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    encodings['labels'] = labels
    return encodings

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Configurare il modello
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Definire gli argomenti di training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.20,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch"
)

for param in model.parameters():
    param.data = param.data.contiguous()

# Definire la funzione di metriche
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision = precision_score(p.label_ids, preds, average='weighted')
    recall = recall_score(p.label_ids, preds, average='weighted')
    f1 = f1_score(p.label_ids, preds, average='weighted')
    accuracy = accuracy_score(p.label_ids, preds)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': accuracy}

# Creare il Trainer con specifica della funzione di perdita
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    compute_metrics=compute_metrics
)

# Addestrare il modello
trainer.train()

# Valutazione del modello sul set di test
metrics = trainer.evaluate(encoded_dataset['test'])
print("Test set evaluation metrics:", metrics)
